In [1]:
import os
from pathlib import Path
import mediapy

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

from pygpudrive.env.config import EnvConfig, RenderConfig, SceneConfig, SelectionDiscipline
from pygpudrive.env.env_torch import GPUDriveTorchEnv

### Configurations

In [2]:
DYNAMICS_MODEL = "state" # "delta_local" or "state"
DATA_PATH = "data/formatted_json_v2_no_tl_train" #processed" 
MAX_NUM_OBJECTS = 128
NUM_ENVS = 20

# Configs
render_config = RenderConfig(draw_obj_idx=True)
scene_config = SceneConfig(path=DATA_PATH, num_scenes=NUM_ENVS, discipline=SelectionDiscipline.PAD_N)
env_config = EnvConfig(dynamics_model=DYNAMICS_MODEL)

In [3]:
env = GPUDriveTorchEnv(
    config=env_config,
    scene_config=scene_config,
    max_cont_agents=MAX_NUM_OBJECTS,
    device="cpu",
    render_config=render_config,
    action_type="continuous" # "continuous" or "discrete"
)


--- Ratio unique scenes / number of worls = 20 / 20 ---



### Get log-playback (expert) actions

- Different dynamics models have different action spaces. For details, [see the docs.](https://github.com/Emerge-Lab/gpudrive/tree/main/pygpudrive/env)

In [4]:
# Extract full expert trajectory
expert_actions, _, _ = env.get_expert_actions()

expert_actions.shape # Shape: (num_envs, num_steps, num_agents, num_actions)

torch.Size([20, 128, 91, 10])

### Step through scene

In [5]:
# Reset environment
obs = env.reset()

frames = {f"env_{i}": [] for i in range(NUM_ENVS)}

# Step through the scene
for t in range(env_config.episode_len):

    env.step_dynamics(expert_actions[:, :, t, :])
    
    # Render the scenes
    for i in range(NUM_ENVS):
        frames[f"env_{i}"].append(env.render(i))

total_goal_achieved = env.get_infos()[env.cont_agent_mask][:, 3].sum()

print(f'Stepping vehicles using their continuous expert trajectories... \n')
print(f'We are controlling {env.cont_agent_mask.sum()} agents in {NUM_ENVS} scenarios. \n')
print(f'Out of these {int(total_goal_achieved)} / {env.cont_agent_mask.sum()} achieved the goal position. \n')

Stepping vehicles using their continuous expert trajectories... 

We are controlling 0 agents in 20 scenarios. 

Out of these 0 / 0 achieved the goal position. 



### Show trajectories

In [6]:
# Show videos
mediapy.show_videos(frames, fps=30, width=600, height=400, columns=4)

env_0 This browser does not support the video tag.,env_1 This browser does not support the video tag.,env_2 This browser does not support the video tag.,env_3 This browser does not support the video tag.
env_4 This browser does not support the video tag.,env_5 This browser does not support the video tag.,env_6 This browser does not support the video tag.,env_7 This browser does not support the video tag.
env_8 This browser does not support the video tag.,env_9 This browser does not support the video tag.,env_10 This browser does not support the video tag.,env_11 This browser does not support the video tag.
env_12 This browser does not support the video tag.,env_13 This browser does not support the video tag.,env_14 This browser does not support the video tag.,env_15 This browser does not support the video tag.
env_16 This browser does not support the video tag.,env_17 This browser does not support the video tag.,env_18 This browser does not support the video tag.,env_19 This browser does not support the video tag.
